# Dispersion Coefficients from Tracer Experiments

The objective of this notebook is to determine a stream's dispersion coefficient from tracer experiment data. Before this notebook is used, the conductivity data must be converted to the concentration of the tracer being injected. In this case, salt dilution gauging techniques were used to determine a regression line between conductivity and sodium chloride (salt) concentration (this was done with excel spreadsheets in the field). After concentration data is inputted into the .csv file, the '.csv' is imported into this notebook. The flow rate of the stream is also determined by calculating the area under this curve and then dividing by the tracer's mass. Then the dispersion coefficient, $K_x$, and stream velocity, $V_x$, are optimized in Taylor's Solution (Eq. 1) (Fischer, 1967; French, 1986, & Rutherford, 1994). <br/> <br/> $C(x,t)=\frac{M}{A\sqrt{4\pi K_x t}} \exp⁡ [\frac{-(x-V_x t)^2}{(4K_x t)}] \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; (Eq. 1)$ <br/> <br/> where $K_x$ is the dispersion coefficient, $V_x$ is the average stream velocity, $A$ is the average cross-sectional area, $M$ is mass of the tracer, $x$ is the distance between source and monitoring station, and $t$ is time.


Populate Notebook with libraries

In [0]:
%pylab inline
%config InlineBackend.figure_format='retina' # hig-res plots for a Retina display
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import pandas as pd
import statsmodels.formula.api as smf
import os
os.chdir('/home/user/Tracer Data') # import working directory for tracer data

#### Input Observed Data and Stream Characteristics

In [0]:
# user inputs ------------------------------------

# data file containing the arrays t_sec and C_mgL
datafile = 'tracer27upM.csv'
data = pd.read_csv(datafile)

t=array(data.t_sec)+1 # 1 second is added to the data to avoid issues with t = 0 in the calcs
C=array(data.C_mgL)

#calculate time of max concentration
tpeak = t[argmax(C)] # time of the peak

# known properties of stream at the time of data collection
Qknown = 0.6476  # flow rate from tracer experiment (ft^3/s)
W1 = 3           # width of stream (ft)
S = 0.0068       # average slope (ft/ft)
M_mg = 1000000   # total mass of salt added (mg)
xknown1 = 289    # distance to source if known (ft)

M = M_mg/1000 # to grams

# convert to SI
ft_m = (30.48/100)
Q = Qknown*ft_m**3
xknown = xknown1*ft_m
x = xknown
W = W1*ft_m

data.head() # preview of the first 5 rows of data loaded

#### Determine average stream velocity, area, and water height

In [0]:
U = x/tpeak # average stream velocity (m/s)
A = Q/U     # average cross-sectional area (m^2/s)
h = A/W     # average water height (m)

### Optimize Dispersion Coef. & Velocity with Taylor Solution.

In [0]:
def taylor2nd(t,K,U): #Taylor Solution as a function of K and U
    Cp = (M/(A*sqrt(4*pi*K*t)))*exp(-(x-U*t)**2/(4*K*t))
    return Cp # returns C in mg/L

# perform the fit w/ initial guesses at minimum (0.01)
# least-squares optimiziation
popt, cov = curve_fit(taylor2nd, t, C, p0=[.1,0.1])

# plot the observed data and the optimized fit using given M, A, and X
fig, ax = plt.subplots()
ax.plot(t/60,C,'o', label='Data')
ax.plot(t/60,taylor2nd(t,popt[0],popt[1]),'--k', label='Taylor Solution')
ax.set_xlabel('Time [min]', fontsize=18)
ax.set_ylabel('Concentration, $C$  [mg/L]', fontsize=18)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)
legend(loc=1, prop={'size': 18})

fig.set_size_inches(10, 8, forward = True)
fig.savefig('tracer27upfit.jpeg',bbox_inches='tight', dpi=100)
show('tracer27upfit.jpeg')

# define 'new' optimized K and U
K = popt[0] # dispersion coefficient
U = popt[1] # averaged cross-secional average velocity throughout stream

print('K optimized [ft^2/s] =', around(K/ft_m**2,decimals = 2))
print('U optimized [ft/s] =', around(U/ft_m,decimals = 2))

### Compute Dispersion Coeff. from Predictor Equations
Predictor equations are simple, practical formulas to determine a dispersion coefficients without performing tracer experiments; however, they can be inaccurate. Dozens of predictor equations have been published. Elder 1959 (Eq. 2), McQuivey & Keefer 1974 (Eq. 3), Iwasa & Aya 1991 (Eq. 4), and Koussis & Rodríguez-Mirasol 1998 (Eq. 5) will be used here. For other predictor equations or selecting a dispersion coefficient from previous studies, Rieckermann et al. 2005 and Wallis and Manson. 2004 are helpful resources referenced following the programs final outputs. <br/> <br/>
$K_E = 5.93u_{star} h \;\;\;\;\;\;\; (Eq. 2) $ <br/> <br/>
$K_M = 0.058\frac{V_x h}{S} \;\;\;\;\;\;\; (Eq. 3) $ <br/> <br/>
$K_I = 2\frac{W}{h}^\frac{3}{2}h u_{star} \;\;\;\;\;\;\; (Eq. 5) $ <br/> <br/>
$K_K = 0.6\frac{W}{h}^2 h u_{star} \;\;\;\;\;\;\; (Eq. 6) $ <br/> <br/>

In [0]:
# compareing optimized K with prediction equation
Pw = W + 2*h #wet perimetter (ft)
Hr = A/Pw #hydraulic radius (ft)
ustar = sqrt(9.81*Hr*S) #shear velocity (ft/s)

# Elder (1959)
KE = 5.93*ustar*h
KE1 = KE/ft_m**2 #(ft^2/s)

# Koussis & Rodríguez-Mirasol 1998
KK = 0.6*(W/h)**2*h*ustar #(m^2/s)
KK1 = KK/ft_m**2 #(ft^2/s)

# Iwasa & Aya 1991
KI = 2*(W/h)**(3/2)*h*ustar #(m^2/s)
KI1 = KI/ft_m**2 #(ft^2/s)

# McQuivey & Keefer 1974
KM = 0.058*(U*h)/S  #(m^2/s)
KM1 = KM/ft_m**2 #(ft^2/s)

In [0]:
# Print information in a 'nice' table
print('-- Stream Characteristics --|-- Customary Units --- | --- SI Units ---')
print('study distance:                  ',around(xknown1, decimals = 3),'  ft          |   ',around(x, decimals = 3), 'm')
print('avg. cross-sectional area:       ',around(A/ft_m**2, decimals = 3),'ft^2      |   ',around(A, decimals = 3), 'm^2')
print('avg. width:                      ',around(W1, decimals = 3),'    ft          |   ',around(W, decimals = 3), 'm')
print('avg. water height:               ',around(h/ft_m, decimals = 3),'ft          |   ',around(h, decimals = 3), 'm')
print('avg. cross-sectional velocity:   ',around(U/ft_m, decimals =3),'ft/s        |   ',around(U, decimals = 3), 'm/s')
print('flowrate:                        ',around(Qknown, decimals =3),'ft^3/s      |   ',around(Q, decimals = 3), 'm^3/s')
print('avg. wet perimeter:              ',around(Pw/ft_m, decimals = 3),'ft          |   ',around(Pw, decimals = 3), 'm')
print('avg. hydraulic radius:           ',around(Hr/ft_m,decimals = 3),'ft          |   ',around(Hr, decimals = 3), 'm')
print('avg. shear velocity:             ',around(ustar/ft_m**2,decimals = 3),'ft^2/s      |   ',around(ustar, decimals = 3), 'm^2/s')
print('')
print('----- Dispersion Coeff. ----|')
print('observed,            K:          ',around(K/ft_m**2, decimals = 3),'ft^2/s      |   ',around(K, decimals = 3),'m^2/s')
print('predictor Elder,    KE:          ',around(KE1, decimals = 3),'ft^2/s      |   ',around(KE, decimals = 3),'m^2/s')
print('predictor Koussis,  KK:          ',around(KK1, decimals = 3),'ft^2/s      |   ',around(KK, decimals = 3),'m^2/s')
print('predictor Iwasa,    KI:          ',around(KI1, decimals = 3),'ft^2/s      |   ',around(KI, decimals = 3),'m^2/s')
print('predictor McQuivey, KM:          ',around(KM1, decimals = 3),'ft^2/s      |   ',around(KM, decimals = 3),'m^2/s')

### References
Elder, J.W. 1959. “The dispresion of marked fluid in turbulaent shear flow.” Journal of Fluid Mechanics 5, no. 4 (May):544-560 <br />
Fischer, H. B. 1967. “The Mechanics of Dispersion in Natural Streams.” American Society of Civil Engineers, 25, No. 5, 62-68 <br />
French, Richard. 1985. “Open-channel hydraulics” McGrawhill, New York. ISSN/ISBN: 0-07-022134-0, 978-0. <br \>
Iwasa, Y., & Shirou Aya, S. 1991. “Predicting longitudinal dispersion coefficeint in open-channel flows.” Proceedings of the International Symposium on Environmental Hydraulics 1, 505-510. <br />
Koussis, A. D. and Jośe Rodríguez-Mirasol. 1998. “Hydraulic Estimation of Dispersion Coefficient for Streams.” Journal of Hydraulic Engineering 124, no. 3:317-320 <br />
McQuivey, R. S. and Thomas N. Keefer. 1974. “Simple Method for Predicting Dispersion in Streams.” Journal of the Environmental Engineering Division 100, no. 4:997-1011. <br />
Rutherford, J. C. 1994. “River Mixing.” John Wiley and Sons Ltd, Chichester, England
